In [1]:
import pytorch_lightning as pl
import pandas as pd
import os
import numpy as np

from src.misc.extracting_runs import extract_runs
import src.utils as utils
from src.data.dataloader import mtlDataModule
from src.pipeline.lightning_frame import mtlMayhemModule

MODEL_HPC = "/work3/s202821/models/"
WORKDIR_HPC = "/zhome/3b/d/154066/repos/multitask-mayhem/"
OUTDIR_HPC = "/work3/s202821/emp_models/"

WORKDIR_LOCAL = "/home/bbejczy/repos/multitask-mayhem/"

os.chdir(WORKDIR_LOCAL)
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
utils.set_seeds()

## Inference time

In [6]:
df_inf = pd.read_csv("reports/figures/inference_time.csv")
df_inf = df_inf.set_index("model")
styler = df_inf.style
styler.format("{:.2f}")
# styler.applymap_index(lambda v: "font-weight: bold;", axis="index")
# styler.applymap_index(lambda v: "font-weight: bold;", axis="columns")
styler.to_latex(
    buf=f"reports/figures/inf-time.tex",
    caption="Inference time comparison on NVIDIA A100-PCI GPU",
    label=f"tab:inf-time",
    position="htbp",
    position_float="centering",
    hrules=True,
    column_format="lccc",
    convert_css=True,
    )


## Separate runs on HPC

In [3]:
# extract_runs(workdir = MODEL_HPC, outdir = OUTDIR_HPC)

In [4]:
os.getcwd()

'/home/bbejczy/repos/multitask-mayhem'

## Run model on test set

In [5]:
df = pd.DataFrame(columns=["name","miou","map","person","forklift","agv","box"])

In [6]:
for root, folders, files in os.walk("reports/test_imgs_all/"):
    # print(folders)
    for file in files:
        if file.endswith(".csv"):
            # print(os.path.join(root, file))
            temp_df = pd.read_csv(os.path.join(root, file))
            df = pd.concat([df,temp_df], ignore_index=True)

In [215]:
# def add_tag(x):
#     tag_list = []
#     if "equal" in x["name"]:
#         tag_list.append("equal")
#     if "dwa" in x["name"]:
#         tag_list.append("dwa")
#     if "v1" in x["name"]:
#         tag_list.append("const")
#     if "uc" in x["name"]:
#         tag_list.append("uncertainty")
#     if "uncertainty" in x["name"]:
#         tag_list.append("uncertainty")
#     if "bmtl" in x["name"]:
#         tag_list.append("bmtl")
#     if "graddrop" in x["name"]:
#         tag_list.append("graddrop")
#     if "cagrad" in x["name"]:
#         tag_list.append("cagrad")
#     if "pcgrad" in x["name"]:
#         tag_list.append("pcgrad")


#     return tag_list


# df["tag"] = df.apply(lambda x: add_tag(x), axis=1)

# df_onehot = pd.DataFrame(columns=["name","miou","map","person","forklift","agv","box","dwa","equal","const","uncertainty","bmtl","graddrop","cagrad","pcgrad"])

# df = pd.concat([df,df_onehot], ignore_index=True)
# df

In [7]:
df.sort_values(by=["map"], inplace=True, ascending=False)
# df["miou"] = (df["person"] + df["forklift"] + df["box"])/3
df

,name,miou,map,person,forklift,agv,box
3,frcnn-hybrid_bmtl_23-02-16T003206,0.897177,0.194977,0.331060,0.081986,0.150000,0.216861
2,frcnn-hybrid_2v1_23-02-21T182529,0.880627,0.190339,0.309146,0.112894,0.100000,0.239316
9,frcnn-hybrid_relative_23-02-16T004645,0.887295,0.189558,0.324733,0.067644,0.166667,0.199187
5,frcnn-hybrid_dwa_23-02-16T234921,0.898418,0.175881,0.325917,0.096141,0.040000,0.241466
14,frcnn-hybrid_uc_23-02-25T204205,0.872598,0.165867,0.363609,0.087618,0.003448,0.208791
1,frcnn-hybrid_10v1_23-02-16T001236,0.858754,0.162731,0.335837,0.087478,0.020000,0.207609
7,frcnn-hybrid_graddrop_23-02-15T235924,0.883271,0.159457,0.307989,0.103246,0.023077,0.203516
8,frcnn-hybrid_pcgrad_23-02-15T155512,0.883751,0.151595,0.285014,0.109731,0.033333,0.178301
6,frcnn-hybrid_equal_23-02-14T235356,0.886795,0.151292,0.285390,0.096690,0.026316,0.196771
4,frcnn-hybrid_cagrad_23-02-15T235953,0.865357,0.141947,0.295353,0.058663,0.000000,0.213771


In [8]:
df["map"] = (df["person"] + df["forklift"] + df["box"])/3
try:
    df.pop("agv")
except:
    pass
# df.pop("agv")
df = df.fillna(0)
df["map"] = df["map"].apply(lambda x:  x*100)
df["person"] = df["person"].apply(lambda x:  x*100)
df["forklift"] = df["forklift"].apply(lambda x:  x*100)
df["box"] = df["box"].apply(lambda x:  x*100)
df["miou"] = df["miou"].apply(lambda x:  x*100)
df.sort_values(by=["map"], inplace=True, ascending=False)
df.set_index("name", inplace=True)
df


styler = df.style

styler.format("{:.2f}")
# styler.applymap_index(lambda v: "font-weight: bold;", axis="index")
# styler.applymap_index(lambda v: "font-weight: bold;", axis="columns")
styler.to_latex(
    buf=f"reports/figures/test-set.tex",
    caption="Placeholder caption",
    label=f"tab:test-set",
    position="H",
    position_float="centering",
    hrules=True,
    column_format="lccccc",
    convert_css=True,
    )

## Extract runs based on tags

In [10]:
import wandb
api = wandb.Api()

In [9]:
# TAG = "singlevsmulti"
# TAG = "weighting"
# TAG = "none-grad"
# TAG = "constant"
# TAG = "constant-grad"
# TAG = "uc-grad"
# TAG = "bmtl-grad"
# TAG = "dwa-grad"
# TAG = "control"

# tags_list = ["singlevsmulti","weighting","none-grad","constant","constant-grad","uc-grad","bmtl-grad","dwa-grad","control"]
tags_list = ["uc-grad"]

In [13]:
def print_tables(tags_list):

    df = pd.read_csv("reports/figures/runs_summary.csv")
    df.Tags = df.Tags.apply(lambda x: x.split(", "))

    df_control_run = pd.DataFrame(columns=["Name", "mAP", "mIoU", "delta mAP", "delta mIoU"])
    control_miou = 95.17
    control_map = 60.36
    df_control_run.Name = ["FRCNN-MTL"]
    df_control_run["mAP"] = [control_map]
    df_control_run["mIoU"] = [control_miou]
    df_control_run["delta mAP"] = [0]
    df_control_run["delta mIoU"] = [0]

    for tag in tags_list:
        print("Tag: ", tag)
        df.Output = df.Tags.apply(lambda x: tag in x)
        df_sampled = df[df.Output][["Name", "ID", "Tags"]]

        df_run_summary = pd.DataFrame(columns=["Name", "mAP", "mIoU", "delta mAP", "delta mIoU"])
        for (_, run_sample) in df_sampled.iterrows():
            print("Run: ", run_sample.Name[:-16])
            run = api.run("/thesis-mayhem/multitask-mayhem/runs/{}".format(run_sample.ID))
            df_run = run.history()
            summary_dict = {"Name": run_sample.Name[:-16], "mIoU": 0, "mAP": 0}

            try:
                summary_dict["mIoU"] = round(df_run["val_loss.seg"].max()*100, 2)
                summary_dict["delta mIoU"] = (summary_dict["mIoU"] - control_miou)
            except:
                pass

            try:
                summary_dict["mAP"] = round(df_run["val_loss.det"].max()*100, 2)
                summary_dict["delta mAP"] = (summary_dict["mAP"] - control_map)
            except:
                pass


            df_run_summary = df_run_summary.append(summary_dict, ignore_index=True)

        df_run_summary = pd.concat([df_control_run, df_run_summary], ignore_index=False)
        df_run_summary = df_run_summary.sort_values(by=["mAP", "mIoU"], ascending=False)
        df_run_summary.set_index("Name", inplace=True)

        styler = df_run_summary.style
        styler.format("{:.2f}")
        # styler.applymap_index(lambda v: "font-weight: bold;", axis="index")
        # styler.applymap_index(lambda v: "font-weight: bold;", axis="columns")
        styler.to_latex(
            buf=f"reports/figures/{tag}.tex",
            caption="Placeholder caption",
            label=f"tab:{tag}",
            position="htbp",
            position_float="centering",
            hrules=True,
            column_format="lcccc",
            convert_css=True,
            )

In [14]:
print_tables(tags_list)

Tag:  uc-grad
Run:  frcnn-hybrid_uc-pcgrad
Run:  frcnn-hybrid_uc-cagrad
Run:  frcnn-hybrid_uc-graddrop
Run:  frcnn-hybrid_uc


In [333]:
df = pd.read_csv("reports/figures/runs_summary.csv")
df.Tags = df.Tags.apply(lambda x: x.split(", "))
df.Output = df.Tags.apply(lambda x: TAG in x)
df_sampled = df[df.Output][["Name", "ID", "Tags"]]
id_list = df_sampled.ID.tolist()
df_sampled

,Name,ID,Tags
5,frcnn-hybrid_bmtl-graddrop_23-02-20T140735,1hx7ni1c,[bmtl-grad]
7,frcnn-hybrid_bmtl-cagrad_23-02-20T132609,15wptbf7,[bmtl-grad]
8,frcnn-hybrid_bmtl-pcgrad_23-02-20T132549,148xg2lj,[bmtl-grad]
18,frcnn-hybrid_bmtl_23-02-16T003206,3ny1ktwq,"[bmtl-grad, bmtl-scale, weighting]"


In [345]:
df.control = df.Tags.apply(lambda x: "control" in x)
df_control = df[df.control][["Name", "ID"]]
df_control

,Name,ID,Tags
30,frcnn-hybrid_equal_23-02-14T235356,3g9znbyd,"[control, equal, none-grad, res, singlevsmulti..."


In [340]:
df.control = df.Tags.apply(lambda x: "control" in x)
df_control = df[df.control][["Name", "ID", "Tags"]]
run = api.run("/thesis-mayhem/multitask-mayhem/runs/{}".format(df_control.ID.item()))
control_miou = round(run.history()[["val_loss.seg"]].max().item()*100,2)
control_map = round(run.history()[["val_loss.det"]].max().item()*100, 2)
print(run.name)
print("Control mIoU: {}%".format(control_miou))
print("Control mAP: {}%".format(control_map))

# control_miou = 95.17
# control_map = 60.36
df_control_run = pd.DataFrame(columns=["Name", "mAP", "mIoU", "delta mAP", "delta mIoU"])
df_control_run.Name = [run.name[:-16]]
df_control_run["mAP"] = [control_map]
df_control_run["mIoU"] = [control_miou]
df_control_run["delta mAP"] = [0]
df_control_run["delta mIoU"] = [0]

frcnn-hybrid_equal_23-02-14T235356
Control mIoU: 94.03%
Control mAP: 60.36%


In [341]:
df_control_run

,Name,mAP,mIoU,delta mAP,delta mIoU
0,frcnn-hybrid_equal,60.36,94.03,0,0


In [335]:
for (header, run_sample) in df_sampled.iterrows():
    print(run_sample.ID)

1hx7ni1c
15wptbf7
148xg2lj
3ny1ktwq


In [336]:
df_run_summary = pd.DataFrame(columns=["Name", "mAP", "mIoU", "delta mAP", "delta mIoU"])
for (_, run_sample) in df_sampled.iterrows():
    run = api.run("/thesis-mayhem/multitask-mayhem/runs/{}".format(run_sample.ID))
    df_run = run.history()
    summary_dict = {"Name": run_sample.Name[:-16], "mIoU": 0, "mAP": 0}

    try:
        summary_dict["mIoU"] = round(df_run["val_loss.seg"].max()*100, 2)
        summary_dict["delta mIoU"] = (summary_dict["mIoU"] - control_miou)
    except:
        pass

    try:
        summary_dict["mAP"] = round(df_run["val_loss.det"].max()*100, 2)
        summary_dict["delta mAP"] = (summary_dict["mAP"] - control_map)
    except:
        pass


    df_run_summary = df_run_summary.append(summary_dict, ignore_index=True)

In [337]:
df_run_summary = pd.concat([df_control_run, df_run_summary], ignore_index=False)
df_run_summary = df_run_summary.sort_values(by=["mAP", "mIoU"], ascending=False)
df_run_summary.set_index("Name", inplace=True)


In [338]:
styler = df_run_summary.style
styler.format("{:.2f}")
# styler.applymap_index(lambda v: "font-weight: bold;", axis="index")
# styler.applymap_index(lambda v: "font-weight: bold;", axis="columns")
styler.to_latex(
    buf=f"reports/figures/{TAG}.tex",
    caption="Placeholder caption",
    label=f"tab:{TAG}",
    position="htbp",
    position_float="centering",
    hrules=True,
    column_format="lcccc",
    convert_css=True,
    )
